In [ ]:
# --- SETUP ---
# conda install -c conda-forge google-cloud-bigquery
from google.cloud import bigquery
from google.oauth2 import service_account
import os


In [2]:
# import global TDM functions
import sys

sys.path.insert(0, "../Resources/2-Python/global-functions")
import BigQuery

client = BigQuery.getBigQueryClient_Confidential2023UtahHTS()


pukar.bhandari
C:/Users/Pukar.Bhandari/.private/confidential-2023-utah-hts-5fd7ddd219a7.json


In [ ]:
# --- CONFIG ---
PROJECT_ID = os.getenv("PROJECT_ID", "wfrc-modeling-data")
BQ_LOCATION = os.getenv("BQ_LOCATION", "US")
TARGET_DATASET = os.getenv("TARGET_DATASET", "prd_tdm_hts_2023")

# Optional prefix/suffix for table names
TABLE_PREFIX = os.getenv("TABLE_PREFIX", "")  # e.g., "build_20251007_"
TABLE_SUFFIX = os.getenv("TABLE_SUFFIX", "")  # e.g., "_v2"

# Ordered list of SQL files to execute
ORDERED_SQL_FILES = [
    "day.sql",
    "hh.sql",
    "person.sql",
    "trip_unlinked.sql",
    "trip_linked.sql",
    "vehicle.sql",
]

# Path to your SQL directory
SQL_DIR = ""  # adjust this path if needed

print(f"✅ Connected to BigQuery project: {PROJECT_ID}")
print(f"Target dataset: {TARGET_DATASET}")
print(f"SQL directory: {SQL_DIR}")


# --- RUNNER FUNCTION ---
def run_sql_file(sql_file: str):
    """Run a SQL file and write results to BigQuery table."""
    base_name = os.path.splitext(sql_file)[0]
    table_name = f"{TABLE_PREFIX}{base_name}{TABLE_SUFFIX}"
    table_id = f"{PROJECT_ID}.{TARGET_DATASET}.{table_name}"

    print(f"\n▶ Running: {sql_file}  →  {table_id}")

    # Load SQL from file
    with open(os.path.join(SQL_DIR, sql_file), "r", encoding="utf-8") as f:
        sql = f.read()

    # Define job config
    job_config = bigquery.QueryJobConfig(
        destination=table_id,
        write_disposition="WRITE_TRUNCATE",  # replace if exists
    )

    # Execute query
    query_job = client.query(sql, job_config=job_config)
    query_job.result()  # Wait for job to complete

    print(f"✔ Done: {table_id}")


# --- MAIN LOOP ---
for file in ORDERED_SQL_FILES:
    file_path = os.path.join(SQL_DIR, file)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Missing file: {file_path}")
    run_sql_file(file)

print("\n✅ All SQL scripts completed successfully.")
